# Heating-Model
For the thermostat website I wanted to be able to simulate how long the furnace would run for. This would be plotted on the webpage to give the user good feeback as to whether the set schedule was optimal or not. This also gave me a good opportunity to apply the calculus, physics, and differential equations. If I spent 4 years learning them I might as well use them. I found a textbook on [Jstor](https://www.jstor.org/stable/2687333) that had a small example similar to what I wanted to do, I used this as a basis. The first two cases are from that textbook. I further iterated upon the model to support the needs of this project. This model takes an average of the insulation for each major component of a home (living space, attic, and roof). It does not calulate the heat transfer rate for each wall nor take into account things like doors and windows. This simplifies the model greatly. Using the data I recorded over a few months, I found values that best fit the model to what I was actually observing. See also the gen_heating_graphs.py in the docs/ folder to see implementation.

## Sub Heading Summary
1. [Bacground](#Background) - How Newton's law of cooling relates to the model.
2. [Setting up the problem](#Setting-up-the-problem) - explains the equations used for the change in temperature per delta time.
3. [Simplfied case](#Simplified-case) - Assumes outside temperature is constant.
4. [A more realistic case](#A-more-realistic-case) - has a changing outside temperature based on a sine wave.
5. [Adding thresholds](#Adding-thresholds) - Allows the furnace to turn on and off during cycles.
6. [Having a set schedule](#Having-a-set-schedule) - Changes the set temperature thoughout the day.
7. [Accounting for sunlight](#Accounting-for-sunlight) - The sun can heat up the home even when the outside temperature is lower than the inside temperature.
8. [Adding heat capacity](#Adding-heat-capacity) - The walls, air and objects in the home have heat capacity, if the target temperature changes to a higher temperature then 
9. [References](#References)

## Background 
Newton's law of cooling describes the rate at which an object changes temperature is proportional to the difference between the object's temperature and the temperature around it. We will use $T(t)$ to denote the outside temperature and the inside temperature as $x(t)$. We will also need a scalling factor for the amount of heat transfered. The heat transfer constant is $k$ which is dependent on the amount and quality of the insulation between the two objects. To describe the values that can be assigned to k, consider the simplified case where the outside temperature is constant with $T(t) = T$. Then, under the substitution $y(t)=x(t)-T$, Newton's law of cooling becomes $y'(t)=-ky(t)$. The solution to this differential equation is the exponential decay equations $y(t)=y_0e^{(-kt)}$. The half-life for such an eaution is $t_{1/2}=(ln2)/k$; that is $t_{1/2}$ is the amount of time it takes for the difference $y(t)=x(t)-T$ in temperatures to reduce to half of its original value. Typical values of $t_{1/2}$ range from 1 for an uninsulated room to 6 for a well-instulated room. Putting this together we get $x'(t)=k[T(t)-x(t)]$. Typical values of $k$ are in the range of $0.115 \approx \frac{ln2}{6} \le k \le \frac{ln2}{1} \approx 0.694$ and $k$ is inversely proportional to the amount of insulation in the walls. For this model we will use $k=0.46,0.35$, and $0.28$ for walls, attic, and roof insulation, respectively. These values correspond approximately to a half life of $1.5$, $2.0$, and $2.5$ hours, respectively [\[1\]](#References)(Sansgiry et al 394).<br>
To simplify this model, heat loss through the floor of the house is ignored. When the furnace is on, it adds heat to the living area at a constant rate $f$ (BTU/hr). If $c$ (degree/hr) is the heat capacity of the air in the living area (the mass of air times the specific heat of air), then if it were not for heat conduction through the walls and ceiling, the temperature of the air in the living area would rise at the constant rate $fc$. Furnaces are generally rated from 60,000 to 100,000 BTU/hr, and the heat capacity of air in a living space is typically in the range of $0.2-0.3\degree F$ per thousand BTU. In our investigation, we will use $f=80,000$ BTU/hr and $c=0.24\degree F$ per thousand BTU, yielding $fc=20\degree F$ per hour. Thus, the rate at which the furnace changes the air temperature in the living area can be described by a step function<br>
<p style="text-align: center;">$H(t)=\begin{cases} 
          20\degree F & \text{when the furnace is on} \\
          0\degree F & \text{when the furnace is off}
       \end{cases}$</p>
       
[\[1\]](#References)(Sansgiry et al 395).
## Setting up the problem
$x_1(t)$ will denote the temperature in the living area at some time $t$, and $x_2(t)$ will denote the temperature in the attic. $H(t)$ denotes the rate at which the furnace raises the living area temperature; $T(t)$ is the outdoor temperature; and $k_1$, $k_2$, and $k_3$ are the heat transfer constants associated with the house's sides, ceiling, and roof, respecively [\[1\]](#References)(Sansgiry et al 395).
![Home Model Image](images/home_model.png)
The living area temperature $x_1(t)$ is affected by the furnace, the outside temperature, and the attic temperature, while the attic temperature $x_2(t)$ is affected by the living area and outdoor temperatures. So by Newton's law of cooling, the rates which the living area and attic temperatures change over time are given by<br>
<p style="text-align: center;">$x'_1(t)=k_1[T(t)-x_1(t)] + k_2[x_2(t)-x_1(t)] + H(t)$</p>
<p style="text-align: center;">$x'_2(t)=k_2[x_1(t)-x_2(t)]+k_3[T(t)-x_2(t)]$</p>

[\[1\]](#References)(Sansgiry et al 396).
### Simplified case
We will initially assume that the sides of the house contain average insulation $(k_1=0.35)$, the ceiling contains minimal insulation $(k_2=0.46)$, and the roof contains above-average insulation $(k_3=0.28)$. With $T(t)=35$ and $H(t)=20$, we observe in Figure 2. that the temperature in the house reaches $68\degree$ after approximately 5.5 hours [\[1\]](#References)(Sansgiry et al 396). Equilibrium is reached at around 74 degrees.
![Simple Case Image](images/simple_case.png)

### A more realistic case
In reality, when the house temperature exceeds some threshold, a thermostat turns off the furnace. We can add a thermostat that turns off the furnace at $68\degree$ by modifying $H(t)$ as follows:
<p style="text-align: center;">$ H(t)=\begin{cases} 
          20 & \text{if } x_1(t) \lt 68 \\
          0 & \text{if } x_1(t) \ge 68
       \end{cases}$</p>
       
To simulate outdoor temperatur fluctuations, we will set
<p style="text-align: center;">
$T(t)=-17.5sin(\frac{\pi}{12}(t+3))+47$</p>

We get the results displayed in Figure 3, which shows that after approximately 7 hours, the thermostat maintains the house temperature at $68\degree$ while the attic temperature fluctuates with the outside temperature [\[1\]](#References)(Sansgiry et al 397). Note that the furnace now turns on and off quite quickly leaving a thicker, more pixelated line.
![Realistic Case Image](images/more_realistic_case.png)

### Adding thresholds
We can further modify $H(t)$ to have thresholds $s_{th}$, such that the furnace will only turn on once $x_1(t)$ falls below the set temperature by a few degrees and then only have the furnace turn off once $x(t)$ exceeds the set temperature by a few degrees. We will also need to maintain the furnace state $F_s$ to know from what side we are approaching  a threshold. Let's say we are cooler than the set temperature and the furnace is on. Eventually we will pass the upper threshold and the furnace will shut off. Soon the temperature will fall below the upper threshold and below the set temperature. But, we don't want the furnace to turn on again until we cross the lower threshold. Applying this logic will keep the furnace from constantly oscillating  from off to on.
<p style="text-align: center;">$ H(t)=\begin{cases} 
          20 & \text{if } x_1(t) \lt (68 - s_{th}) \lor (F_s \land x_1(t) \lt (68 + s_{th})) \\
          0 & \text{if } x_1(t) \ge (68 + s_{th}) 
       \end{cases}$</p>

In Figure 4, you can see that the furnace now turns on for a period and then waits for the lower threshold to be crossed again before turning back on. We can also see that at 3pm there is a longer off period. This shows that the heatloss is less when the temperature difference is smaller.
![Thermostate On Off Case Image](images/thermostat_on_off_case.png)

### Having a set schedule
Now we can make a schedule $S(t)$ that will alter the target temperature $t_t$ throughout the day. At night, we will have it set lower and then turn up higher around noon and then slightly lower for the evening. The schedule function will be as follows
       <p style="text-align: center;">$t_t=S(t)=\begin{cases} 
          60 & \text{if } hour \le 10\\
          70 & \text{if } hour \gt 10 \land  hour \lt 14\\
          68 & \text{if } hour \ge 14
       \end{cases}$</p>
This can then be fed into the new $H(t)$, giving

<p style="text-align: center;">$ H(t)=\begin{cases} 
          20 & \text{if } x_1(t) \lt (t_t - s_{th}) \lor (F_s \land x_1(t) \lt (t_t + s_{th})) \\
          0 & \text{if } x_1(t) \ge (t_t + s_{th}) 
       \end{cases}$</p>
       
Figure 5 shows the result of this schedule. The starting temperatures have been altered to give a more realistic example, assuming the house was warm the day prior. We can see when the schedule jumps to a higher temperature, the furnace turns on and runs for a longer period of time. Then when the schedule drops a bit, the furnace stays off for a longer period.
![Schedule Case Image](images/schedule_case.png)

### Accounting for sunlight
Sunlight that hits the house can play a major role in how long the furnace runs for. Sunlight can heatup the attic and the walls, thus heating up the inside temperature. We can modify our two equations to account for this by adding another term $UV(t)$ with a scaling factor $f_1$ for the walls and $f_2$ for the roof.
<p style="text-align: center;">$x'_1(t)=k_1[T(t)-x_1(t)]+k_2[x_2(t)-x_1(t)]+H(t)+[UV(t)]f_1$</p>
<p style="text-align: center;">$x'_2(t)=k_2[x_1(t)-x_2(t)]+k_3[T(t)-x_2(t)]+[UV(t)]f_2$</p>

Weather data is obtained through an API, in that data is a UV index for the day. These values range from 0 to 10 and as such, must be scaled down. The $H(t)$ function contributes a value of 20 for the examples here, $UV(t)$ (or sunlight) will not be able to contribute that much compared to the furnace. For that reason it needs to be scaled down. I have found that $(f_1=0.2)$ and $(f_2=0.9)$ worked best for me. The highest the UV index gets around my area is 6, making a maximum  of 1.2 and 5.4 respectively.<br>
Figure 6 shows that even though the outside temperature is lower than the inside temperature, the air stays relatively the same and even gains a little bit of heat. The attic increases significantly more as the roof is in constant direct sunlight.
![Sunlight Case Image](images/sunlight_case.png)

### Adding heat capacity
Finally, to make this model more accurate, we need to account for the heat capacity of the air, walls, floors and other objects in the house. If the room is around 60 degrees, for example, and then the set temperature changes to 68 degrees, the air may heat up quickly but other objects will take more time. These cooler objects pull the temperature down more quickly after a temperature jump. Conversely, the same can be said if the temperature is 68 and the schedule jumps to 60, all the objects in the home will take time to lose heat.<br>
To accomplish this, we will take a rolling average of $x_1(t)$ and use $c=[x_{avg} - x_1]$. We can add it to our equation like so
<p style="text-align: center;">$x'_1(t)=k_1[T(t)-x_1(t)]+k_2[x_2(t)-x_1(t)]+H(t)+[UV(t)]f_1+c$</p>

When the average $x_{avg}$ is equal to $x_1(t)$ then this contribution is zero. We keep the average temperature for about the last 2.5 hours. This was estimated by trying different amounts until the model matched what I observed in my home. Now the model responds much more closely to how it does in real life, as shown in Figure 7. Initially, the model resists temperature change and falls slowly, then when the set temperature changes, the temperature reaches the set temperature but quickly falls back down until the house in its entirety reaches the same temperature.

![Heat Capacity Case Image](images/heat_capacity_case.png)


### References
[1] Sansgiry, Prashant S., and Constance C. Edwards. “A Home Heating Model for Calculus Students.” The College Mathematics Journal, vol. 27, no. 5, 1996, pp. 394–97, https://doi.org/10.2307/2687333.
